In [1]:
# !wget https://raw.githubusercontent.com/drabastomek/learningPySpark/master/Chapter03/flight-data/departuredelays.csv
# !$HADOOP_HOME/bin/hdfs dfs -put departuredelays.csv hdfs://hdfs:9000/user/flight

In [2]:
import pyspark
from delta import *
from pyspark.sql.functions import *

In [3]:
warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

builder = pyspark.sql.SparkSession.builder.appName("MyApp").master('spark://master:7077') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", warehouse_location)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a50e45c4-c644-4350-9ccc-71196f615e17;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 615ms :: artifacts dl 24ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 fro

In [4]:
# !$HADOOP_HOME/bin/hdfs dfs -rm -r hdfs://hdfs:9000/user/flight

In [5]:
fulldf = spark.read.format('csv').option('header', True).option('inferSchema', True) \
.load('hdfs://hdfs:9000/user/flight')

In [6]:
fulldf.write.format('delta').mode('overwrite').saveAsTable("default.full_flight")

In [7]:
df = fulldf.limit(10)
df = (df.withColumn('primaryKey', expr("SHA2(concat_ws(',', date,origin,destination),512)")) \
     .withColumn('changeKey', expr("SHA2(concat_ws(',', delay,destination),512)")))
df.write.format('delta').mode('overwrite').saveAsTable('default.small_flight')

In [8]:
df.show()

+-------+-----+--------+------+-----------+--------------------+--------------------+
|   date|delay|distance|origin|destination|          primaryKey|           changeKey|
+-------+-----+--------+------+-----------+--------------------+--------------------+
|1011245|    6|     602|   ABE|        ATL|162b85d8031aee4c0...|d463593f979122fd5...|
|1020600|   -8|     369|   ABE|        DTW|c19de07f14d54bb3c...|3fa1eb583606a3101...|
|1021245|   -2|     602|   ABE|        ATL|0939e101ee38e590b...|bbb9f6468e7533511...|
|1020605|   -4|     602|   ABE|        ATL|74015a471b259ec46...|3b55261eff1f92eb8...|
|1031245|   -4|     602|   ABE|        ATL|66603a6adf37e66bc...|3b55261eff1f92eb8...|
|1030605|    0|     602|   ABE|        ATL|2f6d059b814f81b0e...|1b99e067be9bcbdfa...|
|1041243|   10|     602|   ABE|        ATL|d62deb643c40d25cc...|aa03be7da41aecc61...|
|1040605|   28|     602|   ABE|        ATL|af3f60816d7d9d323...|0b3809e2a4a9e94ea...|
|1051245|   88|     602|   ABE|        ATL|e7b67b7f206

In [9]:
spark.sql('select * from default.small_flight order by primaryKey').show()

+-------+-----+--------+------+-----------+--------------------+--------------------+
|   date|delay|distance|origin|destination|          primaryKey|           changeKey|
+-------+-----+--------+------+-----------+--------------------+--------------------+
|1021245|   -2|     602|   ABE|        ATL|0939e101ee38e590b...|bbb9f6468e7533511...|
|1011245|    6|     602|   ABE|        ATL|162b85d8031aee4c0...|d463593f979122fd5...|
|1030605|    0|     602|   ABE|        ATL|2f6d059b814f81b0e...|1b99e067be9bcbdfa...|
|1050605|    9|     602|   ABE|        ATL|3803f58b13830e879...|e9872741dee48c4d6...|
|1031245|   -4|     602|   ABE|        ATL|66603a6adf37e66bc...|3b55261eff1f92eb8...|
|1020605|   -4|     602|   ABE|        ATL|74015a471b259ec46...|3b55261eff1f92eb8...|
|1040605|   28|     602|   ABE|        ATL|af3f60816d7d9d323...|0b3809e2a4a9e94ea...|
|1020600|   -8|     369|   ABE|        DTW|c19de07f14d54bb3c...|3fa1eb583606a3101...|
|1041243|   10|     602|   ABE|        ATL|d62deb643c4

In [10]:
sql = """
insert into default.small_flight
select date, delay, 999 distance, origin, destination,
SHA2(concat_ws(',', date,origin,destination),512) as primaryKey,
SHA2(concat_ws(',', delay,destination),512) as changeKey
from default.full_flight order by rand() limit 10
"""
spark.sql(sql)

DataFrame[]

In [11]:
spark.sql('select * from default.small_flight order by primaryKey').show()

+-------+-----+--------+------+-----------+--------------------+--------------------+
|   date|delay|distance|origin|destination|          primaryKey|           changeKey|
+-------+-----+--------+------+-----------+--------------------+--------------------+
|1021245|   -2|     602|   ABE|        ATL|0939e101ee38e590b...|bbb9f6468e7533511...|
|1011245|    6|     602|   ABE|        ATL|162b85d8031aee4c0...|d463593f979122fd5...|
|3060730|   -4|     999|   BWI|        STL|17ec14109192b11b7...|f97817e1400219fc1...|
|1030605|    0|     602|   ABE|        ATL|2f6d059b814f81b0e...|1b99e067be9bcbdfa...|
|1050605|    9|     602|   ABE|        ATL|3803f58b13830e879...|e9872741dee48c4d6...|
|1091640|  188|     999|   BMI|        DFW|5ddea3c02874d8dc1...|77898cb32e0ff19ce...|
|1031245|   -4|     602|   ABE|        ATL|66603a6adf37e66bc...|3b55261eff1f92eb8...|
|1161240|    0|     999|   IND|        ORD|689ab3fd001444fa2...|311d56e4551b0b9b8...|
|2040610|    3|     999|   SEA|        LAX|6c0a62ac194

In [12]:
sql = """
insert into default.small_flight
select date, delay, 999 distance, origin, destination,
SHA2(concat_ws(',', date,origin,destination),512) as primaryKey,
SHA2(concat_ws(',', delay,destination),512) as changeKey
from default.small_flight order by rand() limit 10
"""
spark.sql(sql)

DataFrame[]

In [13]:
df = spark.readStream.format('delta').table('default.small_flight')
df

DataFrame[date: int, delay: int, distance: int, origin: string, destination: string, primaryKey: string, changeKey: string]

In [14]:
df = df.withColumn('inserted', current_timestamp())

In [19]:
saveloc = 'hdfs://hdfs:9000/user/flight/stream'

In [21]:
!$HADOOP_HOME/bin/hdfs dfs -rm -r hdfs://hdfs:9000/user/flight/stream

Deleted hdfs://hdfs:9000/user/flight/stream


In [22]:
streamQuery = df.writeStream.format('delta').outputMode('append') \
.option('checkpointLocation', f'{saveloc}/_checkpoint') \
.trigger(once=True) \
.start(saveloc)

22/01/21 12:46:06 WARN MicroBatchExecution: The read limit MaxFiles: 1000 for DeltaSource[hdfs://hdfs:9000/hive/warehouse/small_flight] is ignored when Trigger.Once() is used.


In [24]:
streamQuery.recentProgress

[{'id': 'dc123ca2-da02-45e6-b653-7db332c78ee0',
  'runId': 'c8b3f419-9c21-45e9-a96c-6dc2a34d51a6',
  'name': None,
  'timestamp': '2022-01-21T12:46:06.913Z',
  'batchId': 0,
  'numInputRows': 30,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 5.056463846283499,
  'durationMs': {'addBatch': 3626,
   'getBatch': 14,
   'latestOffset': 2081,
   'queryPlanning': 58,
   'triggerExecution': 5933,
   'walCommit': 94},
  'stateOperators': [],
  'sources': [{'description': 'DeltaSource[hdfs://hdfs:9000/hive/warehouse/small_flight]',
    'startOffset': None,
    'endOffset': {'sourceVersion': 1,
     'reservoirId': '32435ec1-cc7e-4706-8951-bf0a570a4095',
     'reservoirVersion': 14,
     'index': 2,
     'isStartingVersion': True},
    'numInputRows': 30,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 5.056463846283499}],
  'sink': {'description': 'DeltaSink[hdfs://hdfs:9000/user/flight/stream]',
   'numOutputRows': -1}}]

In [25]:
sql = """
drop table if exists default.flightstream;
"""
spark.sql(sql)
sql = f"""
create table default.flightstream using delta location '{saveloc}';
"""
spark.sql(sql)

DataFrame[]

In [26]:
spark.sql('select * from default.flightstream order by primaryKey').show()

+-------+-----+--------+------+-----------+--------------------+--------------------+--------------------+
|   date|delay|distance|origin|destination|          primaryKey|           changeKey|            inserted|
+-------+-----+--------+------+-----------+--------------------+--------------------+--------------------+
|1021245|   -2|     999|   ABE|        ATL|0939e101ee38e590b...|bbb9f6468e7533511...|2022-01-21 12:46:...|
|1021245|   -2|     602|   ABE|        ATL|0939e101ee38e590b...|bbb9f6468e7533511...|2022-01-21 12:46:...|
|1011245|    6|     999|   ABE|        ATL|162b85d8031aee4c0...|d463593f979122fd5...|2022-01-21 12:46:...|
|1011245|    6|     602|   ABE|        ATL|162b85d8031aee4c0...|d463593f979122fd5...|2022-01-21 12:46:...|
|3060730|   -4|     999|   BWI|        STL|17ec14109192b11b7...|f97817e1400219fc1...|2022-01-21 12:46:...|
|3060730|   -4|     999|   BWI|        STL|17ec14109192b11b7...|f97817e1400219fc1...|2022-01-21 12:46:...|
|1030605|    0|     999|   ABE|      

In [27]:
sql = """
insert into default.small_flight
select date, delay, 999 distance, origin, destination,
SHA2(concat_ws(',', date,origin,destination),512) as primaryKey,
SHA2(concat_ws(',', delay,destination),512) as changeKey
from default.small_flight order by rand() limit 10
"""
spark.sql(sql)

DataFrame[]

In [28]:
deltadf = DeltaTable.forName(spark, 'default.flightstream')
deltadf

In [29]:
def merge(microdf, batchId):
    print(f'batch id: {batchId}, rows count: {microdf.count()}')
    microdf = microdf.dropDuplicates(['primaryKey', 'changeKey'])
    deltadf.alias('t').merge(microdf.alias('s'),'s.primaryKey = t.primaryKey') \
    .whenMatchedUpdateAll('s.changeKey <> t.changeKey') \
    .whenNotMatchedInsertAll() \
    .execute()

In [30]:
streamQuery = df.writeStream.format('delta').outputMode('append') \
.foreachBatch(merge) \
.option('checkpointLocation', f'{saveloc}/_checkpoint') \
.trigger(once=True) \
.start(saveloc)

22/01/21 12:46:23 WARN MicroBatchExecution: The read limit MaxFiles: 1000 for DeltaSource[hdfs://hdfs:9000/hive/warehouse/small_flight] is ignored when Trigger.Once() is used.


batch id: 1, rows count: 10


22/01/21 12:46:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [31]:
streamQuery.recentProgress

[]